In [1]:
import torch
import torch.nn as nn
from transformers import AutoModel
import pandas as pd

import gc
import torch as th
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
import numpy as np
import transformers
import transformers as tfs
import torch.optim as optim
import matplotlib.pyplot as plt
from typing import Tuple

torch.manual_seed(0)
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
dtype = torch.bfloat32 if torch.cuda.is_bf16_supported() else \
  torch.float32
print(DEVICE, dtype)

cuda torch.float32


In [2]:
df = pd.read_csv(
  '../input/tabular-playground-series-jun-2022/data.csv',
  index_col=0,
)
sample_sub = pd.read_csv(
  '../input/tabular-playground-series-jun-2022/sample_submission.csv',
  index_col=0
)
# for col in df:
#   if df[col].dtype == 'int64':
#     df[col] = df[col].astype('int32')
#   if df[col].dtype == 'float64':
#     df[col] = df[col].astype('float32')

df = df.astype('float32')

In [3]:
x = df.round(2).values.ravel().astype('float16')

TODO: 
currently, embeddings only cover the float values that happen to be in the dataset. Areas where there are no values, won't get assgned embed values and would therefore be impossible to model to generate. Fix this by instead assigning embeddings to $N$ segments of $\pm3\sigma$ range + 5 extra segments for NaN, $+\infty$, $-\infty$, +extreme, -extreme respectively. 

In [63]:
from sklearn.preprocessing import LabelEncoder
from typing import Tuple
from torch import Tensor
def get_data() -> Tuple[Tensor, LabelEncoder]:
  df = pd.read_csv(
    '../input/tabular-playground-series-jun-2022/data.csv',
    index_col=0,
  )
  # TODO: Make the special values map into 0..10 range
  le = LabelEncoder()
  
  raw_ids = le.fit_transform(x).astype('int16').reshape(df.shape)
  return raw_ids, le

In [5]:
raw_ids, float_encoder = get_data()
# xi = le.inverse_transform(xt)

In [6]:
nan_index = float_encoder.transform([np.nan])[0]
nan_index

3901

In [10]:
class RowDataset(Dataset):
  def __init__(self, data):
    super().__init__()
    self.data = data
  def __len__(self):
    return len(self.data)
  def __getitem__(self,idx):
    return self.data[idx]
  
fullds = RowDataset(raw_ids)
# for x in fullds:
#   print(x)
#   break

BATCH_SIZE = 100
NROWS = df.shape[0]
VALID_NROWS = int(NROWS * .05)
TRAIN_NROWS = NROWS - VALID_NROWS
trainds, valds = random_split(fullds, [TRAIN_NROWS, VALID_NROWS])

traindl = DataLoader(trainds,
                     batch_size=BATCH_SIZE,
                     shuffle=True)
valdl = DataLoader(valds,
                   batch_size=BATCH_SIZE * 2,
                   shuffle=False)
for x in traindl:
  print(x)
  break

tensor([[1790, 1917, 1998,  ..., 1537, 1882, 1821],
        [1878, 1967, 1659,  ..., 1985, 2490, 1778],
        [1951, 1769, 2037,  ..., 1824, 1713, 1952],
        ...,
        [1953, 1793, 1780,  ..., 1977, 2040, 1967],
        [1971, 2025, 2110,  ..., 1922, 1711, 1688],
        [1867, 1949, 2005,  ..., 2534, 2050, 1878]], dtype=torch.int16)


In [11]:
# # x = torch.tensor([1.,2,3])
# # print(x.device)
# # x.to(device)
# # print(x.device)

# x = torch.tensor([1.,2,3])
# print(x.dtype)
# # x = x.to(torch.long)
# x.long()
# print(x.dtype)

In [12]:
# x = nn.CrossEntropyLoss()
# input = torch.randn(3, 5, requires_grad=True)
# target = torch.empty(3, dtype=torch.long).random_(5) - 1
# output = x(input, target)
# input, target, output

TODO: 
- Decide do we only compute loss on masked tokens?
- Do we have that 80/20 chance of random-same token there?
- Seriously consider using tfs DataCollatorForLanguageModeling instead.

In [27]:
df.notna().mean().mean()

0.9875

Loss sources, variants:
- Loss from unmasked only (probably useless, since model has to just memorize stuff it sees, nothing else).
- Loss from masked only (Obv. useful, since this forces the model to guess missing data only)
- Loss from both (Probably the best, since such model would learn faster - and overfit faster as well)

In [58]:
def preprocess(batch_ids: Tensor, 
               mask_proba: Tensor = .2,
               device=DEVICE
              ) -> Tuple[Tensor, Tensor]:
  batch_ids = batch_ids.to(device).long()
  batch_ignored = batch_ids != nan_index
  # Shamelessly stolen from HF data collator for mlm
  proba_matrix = torch.full(batch_ids.shape, 
                            mask_proba,
                            device=device)
  proba_matrix.masked_fill_(batch_ignored, value=.85)
  batch_random_missing_mask = torch.bernoulli(proba_matrix).bool()
  return batch_ids, batch_ignored, batch_random_missing_mask

for batch_ids in traindl:
  break
batch_ids, batch_ignored, batch_random_missing_mask = preprocess(batch_ids, device='cpu')

In [59]:
batch_random_missing_mask.float().mean()

tensor(0.8334)

In [122]:
def get_model(vocab_size = None,
              device='cpu'
             ) -> Tuple[nn.Module, optim.Optimizer]:
  # BUG ALERT: Possbile that embedding or other weights
  # need some scaling. In case model explodes, check it out.
  config = transformers.DistilBertConfig\
    .from_pretrained('hf-internal-testing/tiny-random-distilbert')
  if vocab_size:
    config.vocab_size = vocab_size
  # BUG ALERT: Right now we rely on Sin PE to assign each rounded 
  # value in raveled dataset a separate embedding. 
  config.sinusoidal_pos_embds = True #
  # We will only ever have 80 tokens. Uncomment when all else is
  # working.
  # config.max_position_embeddings = 80
  config.pad_token_id = nan_index
  model = transformers.AutoModelForMaskedLM.from_config(config)
  model.mlm_loss_fct = nn.CrossEntropyLoss(ignore_index=nan_index)
  optim = torch.optim.AdamW(model.parameters(), 
                            lr=1e-5, weight_decay=.1)
  model.to(device)
  return model, optim

In [123]:
config = transformers.DistilBertConfig\
    .from_pretrained('hf-internal-testing/tiny-random-distilbert')


In [124]:
config

DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 32,
  "dropout": 0.1,
  "hidden_act": "gelu",
  "hidden_dim": 37,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 4,
  "n_layers": 5,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "transformers_version": "4.20.1",
  "vocab_size": 1124
}

In [125]:
model, optimizer = get_model(raw_ids.max() + 1)

In [126]:
model

DistilBertForMaskedLM(
  (activation): GELUActivation()
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(3902, 32, padding_idx=3901)
      (position_embeddings): Embedding(512, 32)
      (LayerNorm): LayerNorm((32,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=32, out_features=32, bias=True)
            (k_lin): Linear(in_features=32, out_features=32, bias=True)
            (v_lin): Linear(in_features=32, out_features=32, bias=True)
            (out_lin): Linear(in_features=32, out_features=32, bias=True)
          )
          (sa_layer_norm): LayerNorm((32,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False

In [161]:
with torch.no_grad():
  for batch_ids in traindl:
    batch_ids, batch_ignored, batch_random_missing_mask =\
      preprocess(batch_ids=batch_ids, device='cpu')
    outp = model(
      input_ids=batch_ids,
      labels=batch_ids,
      attention_mask=batch_random_missing_mask,
#       output_attentions = True,
#       output_hidden_states = True,
    )
    print(outp.loss.item())
    break    
    optimizer.zero_grad()
    outp.loss.backward()
    optimizer.step()

#   loss = model.mlm_loss_fct(outp,ignore_index=nan_index)

8.25992202758789


In [154]:
# Check that tweaking pred logits for 
# masked values doesn't affect loss
# def assert_loss_insensitive_to_masked_positions():
#   xhat = outp.logits.view(-1, outp.logits.size(-1)).detach().clone()
#   x = batch_ids.view(-1)
#   before = model.mlm_loss_fct(xhat, x).item()
#   xhat[-1,:] = torch.randn(1,3902)
#   after = model.mlm_loss_fct(xhat, x).item()
#   assert before == after
# assert_loss_insensitive_to_masked_positions()

In [115]:
with torch.no_grad():
  outp = model(
      input_ids=batch_ids,
      labels=batch_ids,
      attention_mask=batch_random_missing_mask,
  #       output_attentions = True,
  #       output_hidden_states = True,
    )

  print(outp.loss.item())

8.278779983520508


In [23]:
outp.loss

tensor(8.2652, grad_fn=<NllLossBackward0>)

In [19]:
outp

MaskedLMOutput(loss=tensor(8.2658), logits=tensor([[[ 0.0000, -0.0131, -0.0718,  ..., -0.0261, -0.3225,  0.1702],
         [ 0.0000,  0.0373, -0.0809,  ...,  0.0625, -0.0432, -0.0620],
         [ 0.0000, -0.0354,  0.2014,  ...,  0.1206, -0.1325,  0.1967],
         ...,
         [ 0.0000,  0.1133,  0.1478,  ..., -0.2031,  0.0664, -0.0747],
         [ 0.0000, -0.0858,  0.0120,  ...,  0.0560,  0.0179,  0.0930],
         [ 0.0000, -0.0532, -0.0503,  ..., -0.0600, -0.1007,  0.0768]],

        [[ 0.0000,  0.0719, -0.1117,  ...,  0.1434,  0.0539, -0.0832],
         [ 0.0000,  0.0947, -0.2859,  ...,  0.1682, -0.0968, -0.0203],
         [ 0.0000,  0.0971,  0.2201,  ...,  0.1498, -0.2391,  0.2101],
         ...,
         [ 0.0000, -0.0046,  0.0448,  ..., -0.1077,  0.0548, -0.0563],
         [ 0.0000,  0.0354, -0.0510,  ...,  0.1373, -0.2650,  0.1154],
         [ 0.0000, -0.0237, -0.2526,  ..., -0.1236,  0.0682, -0.0508]],

        [[ 0.0000, -0.2051, -0.1953,  ...,  0.0466, -0.0691,  0.1230],
  

In [ ]:
STOP

In [ ]:
outp.logits.shape

In [ ]:
outp.keys()

In [ ]:
outp.logits

In [ ]:
aten_mask

In [ ]:
criterion = nn.CrossEntropyLoss(ignore_index=nan_index)

In [ ]:
# # input_ids = torch.randint(0, 1000, (32, 40))
# input_ids = torch.ones(32, 40).int()
# attention_mask = torch.randint(-100,100, (32, 40))
# outp= model(
#   input_ids=input_ids,
#   attention_mask=attention_mask,
#   input_embeds=
#   output_attentions = True,
#   output_hidden_states = True,)
# # [[k,v.shape,v.dtype] for k,v in outp.items()]
# print('Last hiddne state shape', outp.last_hidden_state.shape)
# print('Number of tfm layers + 1 for embed', len(outp.hidden_states)) # 
# print(outp.hidden_states[0].shape)
# print(outp.hidden_states[1].shape)
# print('Attention shape', outp.attentions[0].shape)
# print(len(outp.attentions)) # Number of tfm layers
# outp.keys()

TODO: Issue in torch library - unique returns multiple instances of Na

In [ ]:
model

In [ ]:
raw_data